In [282]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.datasets
from sklearn.ensemble import RandomForestRegressor as rf, GradientBoostingRegressor as gbr, BaggingRegressor as br
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor as knr
from xgboost import XGBRegressor
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn import metrics
pd.set_option("display.max.columns", None)
# pd.set_option("display.max.rows", None)

In [388]:
#storing the data from a file path to the varriable
house_price_train_dataset_path = '/kaggle/input/hse-m-psmsimmo-p-347795-1-apartmentprice/train.csv'
house_price_train_dataset_table = pd.read_csv(house_price_train_dataset_path)
house_price_test_dataset_path = '/kaggle/input/hse-m-psmsimmo-p-347795-1-apartmentprice/test.csv'
house_price_test_dataset_table = pd.read_csv(house_price_test_dataset_path)
(house_price_test_dataset_table. index)
house_price_train_dataset_table.isnull().sum()


Id                 0
Room               0
Price              0
Size               0
Balcony        12750
Floor              0
FloorsTotal        0
Walls           5885
Age              541
Lift            9426
District         357
Okrug            357
dtype: int64

In [389]:
# deleting deletable rows with empty cells if they are not much
house_price_train_dataset_table.dropna(subset=['Age','District','Okrug'], inplace=True)

#Replacing  undeletable rows with empty cells with unknown for those cells
house_price_train_dataset_table = house_price_train_dataset_table.fillna("Unknown")
house_price_test_dataset_table = house_price_test_dataset_table.fillna("Unknown")



In [395]:
# appending new row to train dataset that has values in its cells that are present in test data but not in train beofre creating dummies.
new_row_train = {'Id': 1111111111, 'Room': 0, 'Price': 0.0, 'Size': 0, 'Balcony': 'Unknown', 'Floor': 0, 'FloorsTotal': 0, 'Walls': 'Unknown', 'Age': 'Unknown', 'Lift': 'Unknown', 'District': 'Unknown', 'Okrug': 'Unknown'  }
house_price_train_dataset_table = house_price_train_dataset_table.append(new_row_train, ignore_index=True)

# appending new row to test dataset that has values in its cells that are present in train data but not in train beofre creating dummies.
new_row_test = {'Id': 1111111111, 'Room': 0, 'Size': 0, 'Balcony': 'Unknown', 'Floor': 0, 'FloorsTotal': 0, 'Walls': 'Unknown', 'Age': 'Unknown', 'Lift': 'Unknown', 'District': 'Внуково', 'Okrug': 'СВАО'}
house_price_test_dataset_table = house_price_test_dataset_table.append(new_row_test, ignore_index=True)

# #Deleting ID and the rows created to get the dummies before fedding it to the model
house_price_train_dataset_table = house_price_train_dataset_table[house_price_train_dataset_table.Id != 1111111111]
house_price_test_dataset_table = house_price_test_dataset_table[house_price_test_dataset_table.Id != 1111111111]
house_price_train_dataset_table = house_price_train_dataset_table.drop(['Id'], axis=1)
house_price_test_dataset_table =  house_price_test_dataset_table.drop(['Id'], axis=1)

In [397]:
#undferstanding the correlation between the various features in the data(positive or negative)
# correlation=house_price_train_dataset_table.corr()
# plt.figure(figsize=(20,20))
# sns.heatmap(correlation, cbar=True, square=True, fmt='.1f', annot=True, annot_kws={'size':5}, cmap='Blues')

corr = house_price_train_dataset_table.corr()
print (corr['Price'].sort_values(ascending=False)[120:180])


District_Печатники                -0.02792538
District_Жулебино                 -0.02862440
District_Вешняки                  -0.02865403
District_Рязанский                -0.02912988
District_Соколиная гора           -0.02917823
District_Дмитровский              -0.02955055
District_Южное Чертаново          -0.02976765
District_Южное Орехово Борисово   -0.03035910
District_Люблино                  -0.03071093
District_Гольяново                -0.03371639
District_Отрадное                 -0.03381750
District_Митино                   -0.03401777
District_Бирюлёво Восточное       -0.03461681
District_Марьино                  -0.03856881
Age_новостройка                   -0.04427541
District_Южное Бутово             -0.04442760
Balcony_балкон                    -0.04655042
Okrug_САО                         -0.04677900
Okrug_СЗАО                        -0.04719094
District_Некрасовка               -0.05576818
Okrug_ЮЗАО                        -0.05881235
Walls_блочный                     

In [396]:
# pd.set_option("display.max.rows", None)
# house_price_train_dataset_table.tail()
# house_price_train_dataset_table.head()
# house_price_train_dataset_table.isnull().sum()[0:60]
# house_price_test_dataset_table.shape
# print(house_price_test_dataset_table['Okrug'].unique())
# house_price_train_dataset_table.dtypes
# print(house_price_test_dataset_table.loc[~house_price_test_dataset_table['District'].isin(house_price_train_dataset_table['District']), 'District'])
# print(house_price_test_dataset_table[house_price_test_dataset_table['District'] == 'Внуково'])
# print(house_price_test_dataset_table.columns.difference(house_price_train_dataset_table.columns))

# checking errors with R Sqaured and mean absolute error
# score_1=metrics.r2_score(Y_train, training_data_prediction)
# score_2=metrics.mean_absolute_error(Y_train, training_data_prediction)
# print('R square error: ', score_1)
# print('mean absolute error: ', score_2)

(10127, 150)

In [398]:
# spliting the data
x_train=house_price_train_dataset_table.drop(['Price'], axis=1)
y_train=house_price_train_dataset_table['Price']
x_test=house_price_test_dataset_table
# X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, random_state=2)
# print(X.shape, X_train.shape, X_test.shape )

In [399]:
# training the model
# model = knr()
# model = br()
# model = gbr(learning_rate=0.1, random_state=60)
# model = SVR()
# model=rf(n_estimators = 1000, random_state = 42) #BEST
model=XGBRegressor()
model.fit(x_train, y_train)

RandomForestRegressor(n_estimators=1000, random_state=42)

In [401]:
# Reorder the test dataframe to match the order of the train dataframe
x_test = x_test.reindex(columns = x_train.columns)

# predicting the data
test_data_prediction=model.predict(x_test)
test_data_prediction=pd.DataFrame(test_data_prediction)
pd.options.display.float_format = '{:.8f}'.format
test_data_prediction.index.name = 'Id'
test_data_prediction.index = test_data_prediction.index + 40509
test_data_prediction.rename(columns = {0:'Price'}, inplace = True)
test_data_prediction.to_csv(f"Submission_file_oct-2021-101.csv")
print(test_data_prediction)

                   Price
Id                      
40509   8719565.00000000
40510 227060630.00000000
40511 114670583.66666666
40512 170181760.00000000
40513  39080560.00000000
40514  66813530.00000000
40515  59141730.00000000
40516 112851377.83333333
40517   9961777.00000000
40518   6718029.16666667
40519   9836166.66666667
40520  18148850.83333333
40521   9157045.38095238
40522  12081910.00000000
40523  16275746.66666667
40524   6727625.00000000
40525 124304095.00000000
40526 142040341.83333331
40527   7864574.16666667
40528   6419989.83333333
40529 149345510.00000000
40530 114948290.00000000
40531 129238249.66666664
40532  28149309.33333334
40533   4777904.16666667
40534  19776075.00000000
40535 353665820.00000000
40536   8830537.85714286
40537  19995690.00000000
40538   9259985.00000000
40539 173588320.00000000
40540   7914240.71428571
40541  21459800.00000000
40542  28622431.33333333
40543 109518845.00000000
40544  42208308.33333334
40545  12753026.66666667
40546   8498600.00000000
